In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import pyodbc

In [2]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=prdsql05.westfund.com.au;'
    'DATABASE=BRONZE;'
    'Trusted_Connection=yes;'
)

In [3]:
query = """
SELECT *
  FROM [GOLD].[dbo].[Membership_Reporting]
  WHERE [Run Month Num] = (SELECT MAX([Run Month Num]) FROM [GOLD].[dbo].[Membership_Reporting])
    AND [Product Cover] <> 'Overseas' AND [active_count] <> 0
"""

df_mem = pd.read_sql(query, conn)
len(df_mem)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_37144\720937309.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mem = pd.read_sql(query, conn)


67915

In [4]:
df_mem.head()

,Run Month,Run Month Num,Fin Year,Fin Month,Fin Month Order,MembershipKey,Membership Number,Membership Status,Product Cover,Membership Cover,...,Previous Fund,Join Reason,Termination Code,Termination Reason,Transfered Fund Id,Transfered Fund,active_count,join_count,suspended_count,terminated_count
0,2025-12-31,61,2026,Dec,6,138491PK 2025-12-31,138491.0,A,Combined,Hosp/Extras,...,Medibank,Transfer from another fund,None,None,NaN,None,1,0,0,0
1,2025-12-31,61,2026,Dec,6,138485PK 2025-12-31,138485.0,A,Combined,Hosp/Extras,...,Westfund,Transfer from another fund,None,None,NaN,None,1,0,0,0
2,2025-12-31,61,2026,Dec,6,145861ESA2025-12-31,145861.0,A,Extras only,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
3,2025-12-31,61,2026,Dec,6,138490ESA2025-12-31,138490.0,A,Extras only,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
4,2025-12-31,61,2026,Dec,6,145916PK 2025-12-31,145916.0,A,Combined,Hosp/Extras,...,Westfund,Transfer from another fund,None,None,NaN,None,1,0,0,0


In [5]:
query_2 = """
SELECT DISTINCT
[person_id],
[membership_id]

  FROM [BRONZE].[dbo].[person_membership]

  WHERE [membership_id] IN (SELECT [Membership Number]
  FROM [GOLD].[dbo].[Membership_Reporting]
  WHERE [Run Month Num] = (SELECT MAX([Run Month Num]) FROM [GOLD].[dbo].[Membership_Reporting])
    AND [Product Cover] <> 'Overseas' AND [active_count] <> 0)
"""

df_per_mem = pd.read_sql(query_2, conn)
len(df_per_mem)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_37144\2215566542.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_per_mem = pd.read_sql(query_2, conn)


174864

In [6]:
df_per_mem

,person_id,membership_id
0,386567.0,216603.0
1,165712.0,226426.0
2,417987.0,229022.0
3,359971.0,205419.0
4,411751.0,212301.0
...,...,...
174859,82666.0,74320.0
174860,351249.0,201603.0
174861,379741.0,213990.0
174862,293633.0,126605.0


In [7]:
query = """
SELECT *
  FROM [BRONZE].[dbo].[PersonAddressHomePostal]
"""

df = pd.read_sql(query, conn)
len(df)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_37144\2897101254.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


172388

In [9]:
keys = df[['person_id', 'membership_id']].drop_duplicates()

mask = df_per_mem[['person_id', 'membership_id']].apply(tuple, axis=1).isin(
    keys.apply(tuple, axis=1)
)

df_matched = df_per_mem[mask]
df_unmatched = df_per_mem[~mask]

num_unmatched = (~mask).sum()



In [20]:
len(df_matched)

70384

##### can use [BRONZE].[dbo].[person_membership], [BRONZE].[dbo].[PersonContact],[BRONZE].[dbo].[PersonAddressHomePostal]

In [25]:
df_unmatched
df_unmatched[df_unmatched["membership_id"] == 61319]


,person_id,membership_id
474,51407.0,61319.0
55552,51406.0,61319.0
